In [56]:
import snowflake.connector
import pandas as pd
import numpy as np

In [1]:
account = 'XXXXXXXXXXXXXXXX'
user = 'XXXXXXXXXXXXXXXX'
password = 'XXXXXXXXXXXXXXXX'
database = 'XXXXXXXXXXXXXXXX'
schema = 'XXXXXXXXXXXXXXXX'
warehouse = 'XXXXXXXXXXXXXXXX'

In [3]:
ctx = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    database=database,
    schema=schema,
    warehouse=warehouse
)

In [4]:
account_query = "select * from salesforce_db.raw_data.account"
contact_query = "select * from salesforce_db.raw_data.contact"
opportunity_query = "select * from salesforce_db.raw_data.opportunity"
opportunitystage_query = "select * from salesforce_db.raw_data.opportunitystage"
task_query = "select * from salesforce_db.raw_data.task"

In [5]:
def get_table(ctx, query):
    cs = ctx.cursor()
    try:
        cs.execute(query)
        df = cs.fetch_pandas_all()
    finally:
        cs.close()
    return df

In [6]:
account = get_table(ctx, account_query)

In [7]:
opportunity = get_table(ctx, opportunity_query)

In [8]:
contact = get_table(ctx, contact_query)

In [9]:
opportunitystage = get_table(ctx, opportunitystage_query)

In [10]:
task = get_table(ctx, task_query)

In [12]:
ctx.close()

In [13]:
account.to_csv("raw_data/account.csv", index=False)
opportunity.to_csv("raw_data/opportunity.csv", index=False)
opportunitystage.to_csv("raw_data/opportunitystage.csv", index=False)
task.to_csv("raw_data/task.csv", index=False)
contact.to_csv("raw_data/contact.csv", index=False)

## Functions

In [43]:
def print_missing_data_percentage(df, threshold=50):
    # Calculate the percentage of missing data for each column
    missing_data_percentage = df.isnull().mean() * 100

    # Filter columns with missing data greater than or equal to the threshold
    filtered_columns = missing_data_percentage[missing_data_percentage <= threshold]

    # Print the percentage of missing data for each filtered column
    print(f"Columns with Missing Data Less Than or Equal to {threshold}%:")
    columns = []
    for column, percentage in filtered_columns.sort_values(ascending=False).items():
        print(f"{column}: {percentage:.2f}%")
        columns.append(column)
    return columns

## Account 

In [44]:
account.head()

,ID,ISDELETED,MASTERRECORDID,NAME,TYPE,PARENTID,BILLINGSTREET,BILLINGCITY,BILLINGSTATE,BILLINGPOSTALCODE,...,SLA__C,ACTIVE__C,NUMBEROFLOCATIONS__C,UPSELLOPPORTUNITY__C,SLASERIALNUMBER__C,SLAEXPIRATIONDATE__C,OWNER_INTENT_TO_SELL__C,TOTAL_FUNDING_TO_DATE__C,GROWTH_RATE__C,HQ_LOCATION__C
0,001ak00000IADL0AAP,False,None,Cuenta de ejemplo para asignaciones,None,None,None,None,None,None,...,None,None,None,None,None,None,False,None,None,None
1,001ak00000IADT3AAP,False,None,Edge Communications,Customer - Direct,None,"312 Constitution Place\nAustin, TX 78767\nUSA",Austin,TX,None,...,Silver,Yes,2.0,Maybe,2657,2024-06-12 00:00:00.0000000,False,None,None,None
2,001ak00000IADT4AAP,False,None,Burlington Textiles Corp of America,Customer - Direct,None,525 S. Lexington Ave,Burlington,NC,27215,...,Silver,None,6.0,Maybe,5367,2024-06-12 00:00:00.0000000,False,None,None,None
3,001ak00000IADT7AAP,False,None,Grand Hotels & Resorts Ltd,Customer - Direct,None,"2334 N. Michigan Avenue, Suite 1500\nChicago, ...",Chicago,IL,None,...,Gold,Yes,57.0,Yes,5572,2024-12-13 00:00:00.0000000,False,None,None,None
4,001ak00000IADT8AAP,False,None,United Oil & Gas Corp.,Customer - Direct,None,"1301 Avenue of the Americas \r\nNew York, NY 1...",New York,NY,None,...,Platinum,Yes,955.0,Yes,6654,2024-12-13 00:00:00.0000000,False,None,None,None


In [45]:
# Handling Missing Values
print("Missing Values:")
account_columns = print_missing_data_percentage(account, threshold=20)

Missing Values:
Columns with Missing Data Less Than or Equal to 20%:
HQ_LOCATION__C: 1.96%
GROWTH_RATE__C: 1.96%
TOTAL_FUNDING_TO_DATE__C: 1.96%
YEARSTARTED: 1.96%
ACCOUNTSOURCE: 1.96%
BILLINGLATITUDE: 1.96%
BILLINGLONGITUDE: 1.96%
SITE: 1.96%
BILLINGPOSTALCODE: 1.76%
BILLINGCOUNTRY: 1.76%
ANNUALREVENUE: 0.78%
DESCRIPTION: 0.78%
RATING: 0.59%
TICKERSYMBOL: 0.59%
BILLINGCITY: 0.39%
NUMBEROFEMPLOYEES: 0.20%
OWNERSHIP: 0.20%
INDUSTRY: 0.20%
WEBSITE: 0.20%
FAX: 0.20%
PHONE: 0.20%
TYPE: 0.20%
BILLINGSTREET: 0.20%
BILLINGSTATE: 0.20%
SYSTEMMODSTAMP: 0.00%
NAME: 0.00%
OWNER_INTENT_TO_SELL__C: 0.00%
CLEANSTATUS: 0.00%
LASTMODIFIEDDATE: 0.00%
LASTMODIFIEDBYID: 0.00%
ISDELETED: 0.00%
CREATEDBYID: 0.00%
CREATEDDATE: 0.00%
OWNERID: 0.00%
PHOTOURL: 0.00%
ID: 0.00%


In [51]:
f"# columns: {len(account_columns)}"

'# columns: 36'

In [113]:
account_0 = account[account_columns].copy()
# dropped rows with missing values in the custom fields HQ_LOCATION__C, GROWTH_RATE__C, TOTAL_FUNDING_TO_DATE__C, OWNER_INTENT_TO_SELL__C
account_0.dropna(subset=['HQ_LOCATION__C', 'GROWTH_RATE__C', 'TOTAL_FUNDING_TO_DATE__C', 'OWNER_INTENT_TO_SELL__C'], inplace=True)
# dropped unnecessary columns for now
columns_to_drop = ['BILLINGLATITUDE', 'BILLINGLONGITUDE', 'BILLINGPOSTALCODE', 'BILLINGCOUNTRY', 'DESCRIPTION', 'TICKERSYMBOL',
                   'BILLINGCITY', 'SITE', 'WEBSITE', 'FAX', 'PHONE', 'BILLINGSTREET', 'SYSTEMMODSTAMP', 'CLEANSTATUS', 'BILLINGSTATE',
                   'LASTMODIFIEDDATE', 'LASTMODIFIEDBYID', 'ISDELETED', 'CREATEDBYID', 'CREATEDDATE', 'OWNERID', 'PHOTOURL']
account_0.drop(columns=columns_to_drop, axis=0, inplace=True)

In [115]:
# Check for duplicates
print(f"Duplicates: {account_0.duplicated().sum()}")

Duplicates: 0


In [116]:
account_0.head()

,HQ_LOCATION__C,GROWTH_RATE__C,TOTAL_FUNDING_TO_DATE__C,YEARSTARTED,ACCOUNTSOURCE,ANNUALREVENUE,RATING,NUMBEROFEMPLOYEES,OWNERSHIP,INDUSTRY,TYPE,NAME,OWNER_INTENT_TO_SELL__C,ID
10,Florida,28.0,12618042.5,2024,Incubator/Accelerator Programs,9210250.0,Hot,47,Private,Media,Startup,Flores-Smith Enterprises,True,001ak00000IqvqPAAR
11,South Dakota,240.0,36166195.73,2018,Incubator/Accelerator Programs,27607783,Cold,139,Public,Technology,Growth Stage,"Case, Holden and Shepard Holdings",False,001ak00000IqvqQAAR
12,Alaska,181.0,18847734.44,2016,Direct Referrals,22985042,Cold,115,Public,Recreation,Growth Stage,"Griffin, Rivers and Higgins Enterprises",False,001ak00000IqvqRAAR
13,Indiana,82.0,5496170.0,2022,Networking Events,5496170.0,Warm,28,Subsidiary,Healthcare,Startup,Meza Inc Corporation,False,001ak00000IqvqSAAR
14,New Hampshire,32.0,28215066.73,2015,Business Brokers,47822147,Warm,240,Private,Finance,Growth Stage,Hansen Inc Holdings,True,001ak00000IqvqTAAR


## Opportunity

In [117]:
opportunity.head()

,ID,ISDELETED,ACCOUNTID,ISPRIVATE,NAME,DESCRIPTION,STAGENAME,AMOUNT,PROBABILITY,EXPECTEDREVENUE,...,FISCAL,LASTVIEWEDDATE,LASTREFERENCEDDATE,HASOPENACTIVITY,HASOVERDUETASK,DELIVERYINSTALLATIONSTATUS__C,TRACKINGNUMBER__C,ORDERNUMBER__C,CURRENTGENERATORS__C,MAINCOMPETITORS__C
0,006ak000002sAjJAAU,False,001ak00000IrTIfAAN,False,Investment,Idea south create forget. Rate cost street bet...,Closed Won,9803283.28,100.0,9803283.28,...,4,2017,2017 4,003ak000002nqKoAAI,None,None,True,True,None,None
1,006ak000002sAjKAAU,False,001ak00000IrtTVAAZ,False,Acquisition,Task expert full why organization hair wide. C...,Closed Won,769708262.96,100.0,769708262.96,...,2,2011,2011 2,003ak000002nqKpAAI,None,None,True,True,None,None
2,006ak000002sAjLAAU,False,001ak00000IrtTWAAZ,False,Acquisition,Deal talk sea system single agree recently. Fe...,Closed Lost,15345608.43,0.0,0.0,...,2,2023,2023 2,003ak000002nqKqAAI,None,None,True,True,None,None
3,006ak000002sAjMAAU,False,001ak00000IrtTXAAZ,False,Acquisition,Thought indeed positive fear much almost. Offi...,Closed Lost,14571650.3,0.0,0.0,...,2,2024,2024 2,003ak000002nqKrAAI,None,None,True,True,None,None
4,006ak000002sAjNAAU,False,001ak00000IrtTYAAZ,False,Investment,Though book American. Billion cup win seven.,Closed Lost,53095439.02,0.0,0.0,...,1,2012,2012 1,003ak000002nqKsAAI,None,None,True,True,None,None


In [128]:
opportunity.drop_duplicates(subset=['ID', 'ACCOUNTID'], keep='first', inplace=True)

In [129]:
# Handling Missing Values
print("Missing Values:")
opportunity_columns = print_missing_data_percentage(opportunity, threshold=20)

Missing Values:
Columns with Missing Data Less Than or Equal to 20%:
ID: 0.00%
ISDELETED: 0.00%
TRACKINGNUMBER__C: 0.00%
HASOPENACTIVITY: 0.00%
LASTREFERENCEDDATE: 0.00%
LASTVIEWEDDATE: 0.00%
FISCAL: 0.00%
FISCALQUARTER: 0.00%
SYSTEMMODSTAMP: 0.00%
LASTMODIFIEDBYID: 0.00%
LASTMODIFIEDDATE: 0.00%
CREATEDBYID: 0.00%
CREATEDDATE: 0.00%
OWNERID: 0.00%
HASOPPORTUNITYLINEITEM: 0.00%
FORECASTCATEGORYNAME: 0.00%
FORECASTCATEGORY: 0.00%
ISWON: 0.00%
ISCLOSED: 0.00%
LEADSOURCE: 0.00%
NEXTSTEP: 0.00%
TYPE: 0.00%
CLOSEDATE: 0.00%
EXPECTEDREVENUE: 0.00%
PROBABILITY: 0.00%
AMOUNT: 0.00%
STAGENAME: 0.00%
DESCRIPTION: 0.00%
NAME: 0.00%
ISPRIVATE: 0.00%
ACCOUNTID: 0.00%
ORDERNUMBER__C: 0.00%


In [130]:
f"# columns: {len(opportunity_columns)}"

'# columns: 32'

In [171]:
opportunity_0 = opportunity[opportunity_columns].copy()
# dropped unnecessary columns for now
columns_to_drop = ['ISDELETED', 'FISCAL', 'FISCALQUARTER', 'SYSTEMMODSTAMP', 'LASTMODIFIEDBYID', 'LASTMODIFIEDDATE',
                   'CREATEDBYID', 'CREATEDDATE', 'OWNERID', 'HASOPPORTUNITYLINEITEM', 'HASOPENACTIVITY', 'LASTREFERENCEDDATE',
                   'LASTVIEWEDDATE', 'NEXTSTEP', 'CLOSEDATE', 'DESCRIPTION', 'ISPRIVATE', 'ORDERNUMBER__C']
opportunity_0.drop(columns=columns_to_drop, axis=0, inplace=True)

In [175]:
# Check for duplicates
print(f"Duplicates: {opportunity_0.duplicated().sum()}")

Duplicates: 0


In [232]:
opportunity_0.head()

,ID,TRACKINGNUMBER__C,FORECASTCATEGORYNAME,FORECASTCATEGORY,ISWON,ISCLOSED,LEADSOURCE,TYPE,EXPECTEDREVENUE,PROBABILITY,AMOUNT,STAGENAME,NAME,ACCOUNTID
0,006ak000002sAjJAAU,True,Closed,Closed,True,True,Investment Banks,Existing Business,9803283.28,100.0,9803283.28,Closed Won,Investment,001ak00000IrTIfAAN
1,006ak000002sAjKAAU,True,Closed,Closed,True,True,Direct Referrals,Existing Business,769708262.96,100.0,769708262.96,Closed Won,Acquisition,001ak00000IrtTVAAZ
2,006ak000002sAjLAAU,True,Omitted,Omitted,False,True,Business Brokers,New Business,0.0,0.0,15345608.43,Closed Lost,Acquisition,001ak00000IrtTWAAZ
3,006ak000002sAjMAAU,True,Omitted,Omitted,False,True,Industry Conferences,New Business,0.0,0.0,14571650.3,Closed Lost,Acquisition,001ak00000IrtTXAAZ
4,006ak000002sAjNAAU,True,Omitted,Omitted,False,True,Direct Referrals,Existing Business,0.0,0.0,53095439.02,Closed Lost,Investment,001ak00000IrtTYAAZ


## Contact

In [178]:
contact.head()

,ID,ISDELETED,MASTERRECORDID,ACCOUNTID,LASTNAME,FIRSTNAME,SALUTATION,NAME,OTHERSTREET,OTHERCITY,...,EMAILBOUNCEDREASON,EMAILBOUNCEDDATE,ISEMAILBOUNCED,PHOTOURL,JIGSAW,JIGSAWCONTACTID,CLEANSTATUS,INDIVIDUALID,LEVEL__C,LANGUAGES__C
0,003ak000002hSkfAAE,False,None,001ak00000IADT3AAP,Gonzalez,Rose,Ms.,Rose Gonzalez,None,None,...,None,None,False,/services/images/photo/003ak000002hSkfAAE,None,None,Pending,None,Primary,English
1,003ak000002hSkgAAE,False,None,001ak00000IADT3AAP,Forbes,Sean,Mr.,Sean Forbes,None,None,...,None,None,False,/services/images/photo/003ak000002hSkgAAE,None,None,Pending,None,Secondary,English
2,003ak000002hSkhAAE,False,None,001ak00000IADT4AAP,Rogers,Jack,Mr.,Jack Rogers,None,None,...,None,None,False,/services/images/photo/003ak000002hSkhAAE,None,None,Pending,None,None,None
3,003ak000002hSkkAAE,False,None,001ak00000IADT7AAP,Barr,Tim,Mr.,Tim Barr,None,None,...,None,None,False,/services/images/photo/003ak000002hSkkAAE,None,None,Pending,None,Secondary,English
4,003ak000002hSklAAE,False,None,001ak00000IADT7AAP,Bond,John,Mr.,John Bond,None,None,...,None,None,False,/services/images/photo/003ak000002hSklAAE,None,None,Pending,None,Primary,English


In [181]:
# Handling Missing Values
print("Missing Values:")
contact_columns = print_missing_data_percentage(contact, threshold=20)

Missing Values:
Columns with Missing Data Less Than or Equal to 20%:
HOMEPHONE: 2.91%
DESCRIPTION: 2.91%
MAILINGCITY: 2.72%
MAILINGSTATE: 2.72%
MAILINGCOUNTRY: 2.72%
DEPARTMENT: 0.19%
MOBILEPHONE: 0.19%
BIRTHDATE: 0.19%
PHOTOURL: 0.00%
ISEMAILBOUNCED: 0.00%
SYSTEMMODSTAMP: 0.00%
LASTMODIFIEDBYID: 0.00%
LASTMODIFIEDDATE: 0.00%
CREATEDBYID: 0.00%
CREATEDDATE: 0.00%
OWNERID: 0.00%
ID: 0.00%
TITLE: 0.00%
EMAIL: 0.00%
ISDELETED: 0.00%
FAX: 0.00%
PHONE: 0.00%
MAILINGSTREET: 0.00%
NAME: 0.00%
SALUTATION: 0.00%
FIRSTNAME: 0.00%
LASTNAME: 0.00%
ACCOUNTID: 0.00%
CLEANSTATUS: 0.00%


In [219]:
f"# columns: {len(contact_columns)}"

'# columns: 29'

In [220]:
contact_0 = contact[contact_columns].copy()
# dropped unnecessary columns for now
columns_to_drop = ['HOMEPHONE', 'DESCRIPTION', 'MAILINGCITY', 'MAILINGCOUNTRY', 'MOBILEPHONE', 'BIRTHDATE', 'PHOTOURL', 'ISEMAILBOUNCED',
                   'SYSTEMMODSTAMP', 'LASTMODIFIEDBYID', 'LASTMODIFIEDDATE', 'CREATEDBYID', 'CREATEDDATE', 'OWNERID', 'EMAIL', 'ISDELETED',
                   'FAX', 'PHONE', 'MAILINGSTREET', 'SALUTATION', 'FIRSTNAME', 'LASTNAME', 'CLEANSTATUS']
contact_0.drop(columns=columns_to_drop, axis=0, inplace=True)

# From DEPARTMENT remove some default values 
values_to_drop = ['Production', 'Technology', 'Procurement', 'Facilities', 'Operations', 'Warehouse Mgmt', 'Administration', 'Executive Team']
contact_0 = contact_0[~contact_0['DEPARTMENT'].isin(values_to_drop)]

# From TITLE remove some default values
values_to_drop = ['VP, Facilities', 'VP, Production', 'VP, Technology', 'SVP, Procurement', 'VP, Finance', 'Dean of Administration', 'Director,Warehouse Mgmt',
                  'SVP, Operations', 'SVP, Technology', 'SVP, Production', 'SVP, Administration and Finance', 'Regional General Manager']
contact_0 = contact_0[~contact_0['TITLE'].isin(values_to_drop)]

In [222]:
contact_0.head()

,MAILINGSTATE,DEPARTMENT,ID,TITLE,NAME,ACCOUNTID
1,None,Finance,003ak000002hSkgAAE,CFO,Sean Forbes,001ak00000IADT3AAP
14,None,Finance,003ak000002hSkwAAE,CFO,Avi Green,001ak00000IADT8AAP
15,West Virginia,Other,003ak000002nqKoAAI,VP,Kristy Huffman,001ak00000IrTIfAAN
16,Tennessee,Finance,003ak000002nqKpAAI,Finance Manager,Bethany Frye,001ak00000IrtTVAAZ
17,Louisiana,Finance,003ak000002nqKqAAI,Finance Manager,Lawrence Powers,001ak00000IrtTWAAZ


## Task

In [274]:
task.drop_duplicates(subset=['ID'], keep='first', inplace=True)

In [275]:
task.head()

,ID,WHOID,WHATID,SUBJECT,ACTIVITYDATE,STATUS,PRIORITY,ISHIGHPRIORITY,OWNERID,DESCRIPTION,...,RECURRENCEENDDATEONLY,RECURRENCETIMEZONESIDKEY,RECURRENCETYPE,RECURRENCEINTERVAL,RECURRENCEDAYOFWEEKMASK,RECURRENCEDAYOFMONTH,RECURRENCEINSTANCE,RECURRENCEMONTHOFYEAR,RECURRENCEREGENERATEDTYPE,TASKSUBTYPE
0,00Tak000001SknKEAS,003ak000002nqxxAAA,006ak000002sEVcAAM,Very special discussion treat contain risk.,2023-09-06 00:00:00.0000000,Not Started,Normal,False,005ak000004Jxy9AAC,Little true drive first. Alone partner budget ...,...,None,None,None,None,None,None,None,None,None,Task
1,00Tak000001SknLEAS,003ak000002nqxxAAA,006ak000002sEVcAAM,Turn simply what day goal increase.,2023-09-06 00:00:00.0000000,Not Started,Low,False,005ak000004Jxy9AAC,Ability serve case together air build. Positiv...,...,None,None,None,None,None,None,None,None,None,Call
2,00Tak000001SknMEAS,003ak000002nqxxAAA,006ak000002sEVcAAM,Lose western table card just realize simple.,2023-09-06 00:00:00.0000000,Not Started,High,True,005ak000004Jxy9AAC,Debate much region main record. Vote remain bl...,...,None,None,None,None,None,None,None,None,None,Email
3,00Tak000001SknNEAS,003ak000002nqxxAAA,006ak000002sEVcAAM,Large friend human art yard hour.,2023-09-06 00:00:00.0000000,Completed,High,True,005ak000004Jxy9AAC,Everyone oil election single eight. Street occ...,...,None,None,None,None,None,None,None,None,None,Email
4,00Tak000001SknOEAS,003ak000002nqxyAAA,006ak000002sEVdAAM,Avoid claim lot factor.,2022-07-31 00:00:00.0000000,Not Started,Low,False,005ak000004Jxy9AAC,Exactly capital the century heavy box would. L...,...,None,None,None,None,None,None,None,None,None,Email


In [276]:
# Handling Missing Values
print("Missing Values:")
task_columns = print_missing_data_percentage(task, threshold=20)

Missing Values:
Columns with Missing Data Less Than or Equal to 20%:
ID: 0.00%
WHOID: 0.00%
ISRECURRENCE: 0.00%
ISREMINDERSET: 0.00%
CALLOBJECT: 0.00%
CALLDISPOSITION: 0.00%
CALLTYPE: 0.00%
CALLDURATIONINSECONDS: 0.00%
ISARCHIVED: 0.00%
SYSTEMMODSTAMP: 0.00%
LASTMODIFIEDBYID: 0.00%
LASTMODIFIEDDATE: 0.00%
CREATEDBYID: 0.00%
CREATEDDATE: 0.00%
ISCLOSED: 0.00%
ACCOUNTID: 0.00%
ISDELETED: 0.00%
DESCRIPTION: 0.00%
OWNERID: 0.00%
ISHIGHPRIORITY: 0.00%
PRIORITY: 0.00%
STATUS: 0.00%
ACTIVITYDATE: 0.00%
SUBJECT: 0.00%
WHATID: 0.00%
TASKSUBTYPE: 0.00%


In [277]:
f"# columns: {len(task_columns)}"

'# columns: 26'

In [278]:
task_0 = task[task_columns].copy()
# dropped unnecessary columns for now
columns_to_drop = ['ISRECURRENCE', 'ISREMINDERSET', 'CALLOBJECT', 'ISARCHIVED', 'SYSTEMMODSTAMP', 'LASTMODIFIEDBYID', 'SUBJECT',
                   'LASTMODIFIEDDATE', 'CREATEDBYID', 'CREATEDDATE', 'ISDELETED', 'DESCRIPTION', 'OWNERID', 'ACTIVITYDATE']
task_0.drop(columns=columns_to_drop, axis=0, inplace=True)

In [279]:
task_0.columns

Index(['ID', 'WHOID', 'CALLDISPOSITION', 'CALLTYPE', 'CALLDURATIONINSECONDS',
       'ISCLOSED', 'ACCOUNTID', 'ISHIGHPRIORITY', 'PRIORITY', 'STATUS',
       'WHATID', 'TASKSUBTYPE'],
      dtype='object')

In [280]:
task_0.head()

,ID,WHOID,CALLDISPOSITION,CALLTYPE,CALLDURATIONINSECONDS,ISCLOSED,ACCOUNTID,ISHIGHPRIORITY,PRIORITY,STATUS,WHATID,TASKSUBTYPE
0,00Tak000001SknKEAS,003ak000002nqxxAAA,Interested - Follow Up Needed,Internal,68,False,001ak00000IruOBAAZ,False,Normal,Not Started,006ak000002sEVcAAM,Task
1,00Tak000001SknLEAS,003ak000002nqxxAAA,Not Interested,Outbound,45,False,001ak00000IruOBAAZ,False,Low,Not Started,006ak000002sEVcAAM,Call
2,00Tak000001SknMEAS,003ak000002nqxxAAA,Not Interested,Inbound,39,False,001ak00000IruOBAAZ,True,High,Not Started,006ak000002sEVcAAM,Email
3,00Tak000001SknNEAS,003ak000002nqxxAAA,Call Successful,Inbound,348,True,001ak00000IruOBAAZ,True,High,Completed,006ak000002sEVcAAM,Email
4,00Tak000001SknOEAS,003ak000002nqxyAAA,Scheduled Call Back,Outbound,38,False,001ak00000IruOCAAZ,False,Low,Not Started,006ak000002sEVdAAM,Email


## Saving the Prepro Data

In [293]:
account_0.to_csv('preprod_data/account.csv', index=False)
opportunity_0.to_csv('preprod_data/opportunity.csv', index=False)
contact_0.to_csv('preprod_data/contact.csv', index=False)
task_0.to_csv('preprod_data/task.csv', index=False)

In [3]:
ds = pd.read_csv('preprod_data/account.csv')

In [7]:
ds.TYPE.value_counts()

TYPE
Established     271
Startup         161
Growth Stage     68
Name: count, dtype: int64

In [8]:
os = pd.read_csv('preprod_data/opportunity.csv')

In [13]:
os.STAGENAME.value_counts()

STAGENAME
Closed Lost             287
Closed Won               66
Value Proposition        29
Prospecting              25
Needs Analysis           23
Qualification            18
Negotiation/Review       15
Proposal/Price Quote     13
Perception Analysis      13
Id. Decision Makers      11
Name: count, dtype: int64

In [12]:
os

,ID,TRACKINGNUMBER__C,FORECASTCATEGORYNAME,FORECASTCATEGORY,ISWON,ISCLOSED,LEADSOURCE,TYPE,EXPECTEDREVENUE,PROBABILITY,AMOUNT,STAGENAME,NAME,ACCOUNTID
0,006ak000002sAjJAAU,True,Closed,Closed,True,True,Investment Banks,Existing Business,9.803283e+06,100.0,9.803283e+06,Closed Won,Investment,001ak00000IrTIfAAN
1,006ak000002sAjKAAU,True,Closed,Closed,True,True,Direct Referrals,Existing Business,7.697083e+08,100.0,7.697083e+08,Closed Won,Acquisition,001ak00000IrtTVAAZ
2,006ak000002sAjLAAU,True,Omitted,Omitted,False,True,Business Brokers,New Business,0.000000e+00,0.0,1.534561e+07,Closed Lost,Acquisition,001ak00000IrtTWAAZ
3,006ak000002sAjMAAU,True,Omitted,Omitted,False,True,Industry Conferences,New Business,0.000000e+00,0.0,1.457165e+07,Closed Lost,Acquisition,001ak00000IrtTXAAZ
4,006ak000002sAjNAAU,True,Omitted,Omitted,False,True,Direct Referrals,Existing Business,0.000000e+00,0.0,5.309544e+07,Closed Lost,Investment,001ak00000IrtTYAAZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,006ak000002sEZQAA2,True,Pipeline,Pipeline,False,False,Investment Banks,Existing Business,8.059185e+07,70.0,1.151312e+08,Perception Analysis,Investment,001ak00000IruJwAAJ
496,006ak000002sEZRAA2,True,Omitted,Omitted,False,True,Networking Events,Existing Business,0.000000e+00,0.0,2.245262e+08,Closed Lost,Acquisition,001ak00000IruJxAAJ
497,006ak000002sEZSAA2,True,Pipeline,Pipeline,False,False,Networking Events,Existing Business,1.099439e+09,75.0,1.465919e+09,Proposal/Price Quote,Acquisition,001ak00000IruJyAAJ
498,006ak000002sEZTAA2,True,Omitted,Omitted,False,True,Direct Referrals,Existing Business,0.000000e+00,0.0,1.249053e+09,Closed Lost,Acquisition,001ak00000IruJzAAJ


In [14]:
cc = pd.read_csv('preprod_data/contact.csv')

In [16]:
cc.ACCOUNTID.value_counts()

ACCOUNTID
001ak00000IADT3AAP    1
001ak00000IruOCAAZ    1
001ak00000IqvqUAAR    1
001ak00000IqvqTAAR    1
001ak00000IqvqSAAR    1
                     ..
001ak00000IruLBAAZ    1
001ak00000IruLAAAZ    1
001ak00000IruL9AAJ    1
001ak00000IrTIUAA3    1
001ak00000IruK0AAJ    1
Name: count, Length: 502, dtype: int64

In [17]:
opportunity = pd.read_csv('preprod_data/opportunity.csv')
task = pd.read_csv('preprod_data/task.csv')

In [19]:
opportunity.head()

,ID,TRACKINGNUMBER__C,FORECASTCATEGORYNAME,FORECASTCATEGORY,ISWON,ISCLOSED,LEADSOURCE,TYPE,EXPECTEDREVENUE,PROBABILITY,AMOUNT,STAGENAME,NAME,ACCOUNTID
0,006ak000002sAjJAAU,True,Closed,Closed,True,True,Investment Banks,Existing Business,9.803283e+06,100.0,9.803283e+06,Closed Won,Investment,001ak00000IrTIfAAN
1,006ak000002sAjKAAU,True,Closed,Closed,True,True,Direct Referrals,Existing Business,7.697083e+08,100.0,7.697083e+08,Closed Won,Acquisition,001ak00000IrtTVAAZ
2,006ak000002sAjLAAU,True,Omitted,Omitted,False,True,Business Brokers,New Business,0.000000e+00,0.0,1.534561e+07,Closed Lost,Acquisition,001ak00000IrtTWAAZ
3,006ak000002sAjMAAU,True,Omitted,Omitted,False,True,Industry Conferences,New Business,0.000000e+00,0.0,1.457165e+07,Closed Lost,Acquisition,001ak00000IrtTXAAZ
4,006ak000002sAjNAAU,True,Omitted,Omitted,False,True,Direct Referrals,Existing Business,0.000000e+00,0.0,5.309544e+07,Closed Lost,Investment,001ak00000IrtTYAAZ


In [20]:
task.head()

,ID,WHOID,CALLDISPOSITION,CALLTYPE,CALLDURATIONINSECONDS,ISCLOSED,ACCOUNTID,ISHIGHPRIORITY,PRIORITY,STATUS,WHATID,TASKSUBTYPE
0,00Tak000001SknKEAS,003ak000002nqxxAAA,Interested - Follow Up Needed,Internal,68,False,001ak00000IruOBAAZ,False,Normal,Not Started,006ak000002sEVcAAM,Task
1,00Tak000001SknLEAS,003ak000002nqxxAAA,Not Interested,Outbound,45,False,001ak00000IruOBAAZ,False,Low,Not Started,006ak000002sEVcAAM,Call
2,00Tak000001SknMEAS,003ak000002nqxxAAA,Not Interested,Inbound,39,False,001ak00000IruOBAAZ,True,High,Not Started,006ak000002sEVcAAM,Email
3,00Tak000001SknNEAS,003ak000002nqxxAAA,Call Successful,Inbound,348,True,001ak00000IruOBAAZ,True,High,Completed,006ak000002sEVcAAM,Email
4,00Tak000001SknOEAS,003ak000002nqxyAAA,Scheduled Call Back,Outbound,38,False,001ak00000IruOCAAZ,False,Low,Not Started,006ak000002sEVdAAM,Email


In [26]:
ds = opportunity.merge(task, left_on="ID", right_on="WHATID", how="left")

In [37]:
ds = ds[ds.STAGENAME.isin(['Closed Won', 'Closed Lost'])].copy()

In [27]:
pd.pivot_table(ds, index="index_col", values="value_col", aggfunc=aggfunc)

,ID_x,TRACKINGNUMBER__C,FORECASTCATEGORYNAME,FORECASTCATEGORY,ISWON,ISCLOSED_x,LEADSOURCE,TYPE,EXPECTEDREVENUE,PROBABILITY,...,CALLDISPOSITION,CALLTYPE,CALLDURATIONINSECONDS,ISCLOSED_y,ACCOUNTID_y,ISHIGHPRIORITY,PRIORITY,STATUS,WHATID,TASKSUBTYPE
0,006ak000002sAjJAAU,True,Closed,Closed,True,True,Investment Banks,Existing Business,9803283.28,100.0,...,Call Successful,Internal,314,True,001ak00000IrTIfAAN,False,Normal,Completed,006ak000002sAjJAAU,Task
1,006ak000002sAjJAAU,True,Closed,Closed,True,True,Investment Banks,Existing Business,9803283.28,100.0,...,No Answer,Inbound,57,False,001ak00000IrTIfAAN,True,High,Not Started,006ak000002sAjJAAU,Call
2,006ak000002sAjJAAU,True,Closed,Closed,True,True,Investment Banks,Existing Business,9803283.28,100.0,...,Call Successful,Outbound,598,True,001ak00000IrTIfAAN,False,Normal,Completed,006ak000002sAjJAAU,Task
3,006ak000002sAjJAAU,True,Closed,Closed,True,True,Investment Banks,Existing Business,9803283.28,100.0,...,Interested - Follow Up Needed,Internal,91,False,001ak00000IrTIfAAN,False,Low,Not Started,006ak000002sAjJAAU,Task
4,006ak000002sAjJAAU,True,Closed,Closed,True,True,Investment Banks,Existing Business,9803283.28,100.0,...,Call Successful,Outbound,355,True,001ak00000IrTIfAAN,True,High,Completed,006ak000002sAjJAAU,Call
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519,006ak000002sEZTAA2,True,Omitted,Omitted,False,True,Direct Referrals,Existing Business,0.00,0.0,...,Interested - Follow Up Needed,Inbound,74,False,001ak00000IruJzAAJ,True,High,Not Started,006ak000002sEZTAA2,Task
1520,006ak000002sEZUAA2,True,Omitted,Omitted,False,True,Industry Conferences,Existing Business,0.00,0.0,...,Not Interested,Internal,34,False,001ak00000IruK0AAJ,False,Low,Not Started,006ak000002sEZUAA2,Email
1521,006ak000002sEZUAA2,True,Omitted,Omitted,False,True,Industry Conferences,Existing Business,0.00,0.0,...,Call Successful,Inbound,503,True,001ak00000IruK0AAJ,True,High,Completed,006ak000002sEZUAA2,Email
1522,006ak000002sEZUAA2,True,Omitted,Omitted,False,True,Industry Conferences,Existing Business,0.00,0.0,...,Interested - Follow Up Needed,Internal,99,False,001ak00000IruK0AAJ,True,High,Not Started,006ak000002sEZUAA2,Email


In [28]:
opportunity.columns

Index(['ID', 'TRACKINGNUMBER__C', 'FORECASTCATEGORYNAME', 'FORECASTCATEGORY',
       'ISWON', 'ISCLOSED', 'LEADSOURCE', 'TYPE', 'EXPECTEDREVENUE',
       'PROBABILITY', 'AMOUNT', 'STAGENAME', 'NAME', 'ACCOUNTID'],
      dtype='object')

In [35]:
opportunity.STAGENAME.value_counts()

STAGENAME
Closed Lost             287
Closed Won               66
Value Proposition        29
Prospecting              25
Needs Analysis           23
Qualification            18
Negotiation/Review       15
Proposal/Price Quote     13
Perception Analysis      13
Id. Decision Makers      11
Name: count, dtype: int64

In [38]:
ds.STAGENAME.value_counts()

STAGENAME
Closed Lost    691
Closed Won     309
Name: count, dtype: int64

In [60]:
callduration = task.groupby(['WHATID', 'TASKSUBTYPE']).sum().reset_index()[['WHATID', 'TASKSUBTYPE', 'CALLDURATIONINSECONDS']]
callduration['CALLDURATIONINSECONDS'] = callduration['CALLDURATIONINSECONDS'].astype(float)

In [68]:
call_duration_df = callduration[callduration['TASKSUBTYPE']=='Call'][['WHATID', 'CALLDURATIONINSECONDS']].groupby(['WHATID']).agg('mean').reset_index()

In [48]:
task.groupby(['WHATID', 'TASKSUBTYPE']).count().reset_index()[['WHATID', 'TASKSUBTYPE', 'STATUS']]

,WHATID,TASKSUBTYPE,STATUS
0,006ak000002sAjJAAU,Call,3
1,006ak000002sAjJAAU,Task,3
2,006ak000002sAjKAAU,Call,4
3,006ak000002sAjKAAU,Email,1
4,006ak000002sAjLAAU,Call,1
...,...,...,...
907,006ak000002sEZSAA2,Call,2
908,006ak000002sEZSAA2,Email,2
909,006ak000002sEZTAA2,Task,1
910,006ak000002sEZUAA2,Email,3


In [ ]:
task.TASKSUBTYPE

In [44]:
task.head()

,ID,WHOID,CALLDISPOSITION,CALLTYPE,CALLDURATIONINSECONDS,ISCLOSED,ACCOUNTID,ISHIGHPRIORITY,PRIORITY,STATUS,WHATID,TASKSUBTYPE
0,00Tak000001SknKEAS,003ak000002nqxxAAA,Interested - Follow Up Needed,Internal,68,False,001ak00000IruOBAAZ,False,Normal,Not Started,006ak000002sEVcAAM,Task
1,00Tak000001SknLEAS,003ak000002nqxxAAA,Not Interested,Outbound,45,False,001ak00000IruOBAAZ,False,Low,Not Started,006ak000002sEVcAAM,Call
2,00Tak000001SknMEAS,003ak000002nqxxAAA,Not Interested,Inbound,39,False,001ak00000IruOBAAZ,True,High,Not Started,006ak000002sEVcAAM,Email
3,00Tak000001SknNEAS,003ak000002nqxxAAA,Call Successful,Inbound,348,True,001ak00000IruOBAAZ,True,High,Completed,006ak000002sEVcAAM,Email
4,00Tak000001SknOEAS,003ak000002nqxyAAA,Scheduled Call Back,Outbound,38,False,001ak00000IruOCAAZ,False,Low,Not Started,006ak000002sEVdAAM,Email


In [69]:
call_duration_df

,WHATID,CALLDURATIONINSECONDS
0,006ak000002sAjJAAU,515.0
1,006ak000002sAjKAAU,621.0
2,006ak000002sAjLAAU,30.0
3,006ak000002sAjMAAU,564.0
4,006ak000002sAjQAAU,90.0
...,...,...
247,006ak000002sEZGAA2,256.0
248,006ak000002sEZHAA2,350.0
249,006ak000002sEZIAA2,143.0
250,006ak000002sEZQAA2,84.0


In [73]:
something = opportunity.merge(call_duration_df, left_on='ID', right_on='WHATID', how='left')[['STAGENAME', 'CALLDURATIONINSECONDS']]

In [75]:
something = something[something.STAGENAME.isin(['Closed Won', 'Closed Lost'])].copy()

In [78]:
something['CALLDURATIONINSECONDS'] = something['CALLDURATIONINSECONDS'].fillna(0)

In [79]:
something

,STAGENAME,CALLDURATIONINSECONDS
0,Closed Won,515.0
1,Closed Won,621.0
2,Closed Lost,30.0
3,Closed Lost,564.0
4,Closed Lost,0.0
...,...,...
492,Closed Lost,0.0
493,Closed Lost,0.0
496,Closed Lost,0.0
498,Closed Lost,0.0


In [80]:
pd.pivot_table(something, index="STAGENAME", values="CALLDURATIONINSECONDS")

,CALLDURATIONINSECONDS
STAGENAME,
Closed Lost,52.066202
Closed Won,533.090909
